In [1]:
from geopandas import GeoDataFrame
from shapely.geometry import Point
import geemap
import ee
import os
import geopandas as gpd

from shapely import wkt
import pandas as pd
import matplotlib.pyplot as plt

### Funciones

In [2]:
def calcular_distancia(imagen_pixel, nom_vars_list, df_sh_list):
    
    """Funcion para calcular la distancia de la imagen con diferentes sectores (medido en metros)"""
    
    for i in range(len(df_sh_list)):
        print('revisando ' + str(i) + '----')
        df_sh_list[i].to_crs(epsg=32719, inplace=True)                # to meters
        imagen_pixel[nom_vars_list[i]] = imagen_pixel.geometry.apply(lambda x: df_sh_list[i].distance(x).min())  # add distance colums
    
    return imagen_pixel

def add_time(image):
    """Funcion para realizar un filtro de fechas con map"""
    return image.set('Date', image.date().format("YYYY-MM-dd"))



def calculate_weather(image, fecha, df_4326):
    
    # filtro df
    df_aux = df_4326[df_4326['time2'] == fecha] # global env
    
    # recolectamos los puntos
    points = df_aux.geometry

    # formato apto para buscar los puntos en gee
    jsonDict = eval(points.to_json()) 
    fc = ee.FeatureCollection(jsonDict)
    imagen_per_point = image.sampleRegions(collection = fc)
    
    fc_point_samp = geemap.ee_to_df(imagen_per_point)
    df_aux = df_aux.reset_index(drop=True)
    fc_point_samp = fc_point_samp.reset_index(drop=True)
    
    return pd.concat([df_aux, fc_point_samp], axis=1)


### Estimando distancias

In [3]:
# data = pd.read_csv('muestra_datos_consolidados_v2.csv')
# data = pd.read_csv('../model/data/valparaiso/datos_consolidados_sentinel2.csv')
data = pd.read_csv('../model/data/valparaiso/datos_consolidados_sentinel2_v4.csv')
data

,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,codprov,codcom,nom_incen,causa,superficie,fecha_ini,fecha_ter,tipo_dato,id_pixel,id_pixel_incendio
0,0,2022-02-02,0,0.1324,0.1278,0.1418,0.1498,0.2619,0.2929,0.2298,...,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,1,2021-2022-3-1
1,1,2022-02-02,1,0.1313,0.1252,0.1425,0.1383,0.2820,0.2676,0.2038,...,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,2,2021-2022-3-2
2,2,2022-02-02,2,0.1388,0.1322,0.1465,0.1517,0.2457,0.2914,0.2357,...,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,3,2021-2022-3-3
3,3,2022-02-02,3,0.1325,0.1284,0.1394,0.1420,0.2357,0.2532,0.2015,...,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,4,2021-2022-3-4
4,4,2022-02-02,4,0.1350,0.1376,0.1542,0.1680,0.2580,0.3237,0.2530,...,56,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,5,2021-2022-3-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202259,202259,2020-12-24,2985,0.0477,0.0715,0.1014,0.1504,0.2394,0.3648,0.2752,...,58,5801,151 - LA FRAGUA,2.1.11. Otros intencionales no clasificados,348.660,2020-12-15,2020-12-23,zona_random,53339,2020-2021-11-53339
202260,202260,2020-12-24,2986,0.0613,0.0895,0.1366,0.2058,0.2512,0.2692,0.2359,...,58,5801,151 - LA FRAGUA,2.1.11. Otros intencionales no clasificados,348.660,2020-12-15,2020-12-23,zona_random,53340,2020-2021-11-53340
202261,202261,2020-12-24,2988,0.0606,0.0809,0.1080,0.1515,0.2329,0.3211,0.2456,...,58,5801,151 - LA FRAGUA,2.1.11. Otros intencionales no clasificados,348.660,2020-12-15,2020-12-23,zona_random,53341,2020-2021-11-53341
202262,202262,2020-12-24,2994,0.0494,0.0539,0.0757,0.0869,0.2506,0.2643,0.1827,...,58,5801,151 - LA FRAGUA,2.1.11. Otros intencionales no clasificados,348.660,2020-12-15,2020-12-23,zona_random,53342,2020-2021-11-53342


In [4]:
geometry = [Point(xy) for xy in zip(data.x, data.y)]
# df = muestra_datos_consolidados_v2.drop(['x', 'y'], axis=1)
df = data
df = GeoDataFrame(df, crs="EPSG:32719", geometry=geometry)
df

,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,codcom,nom_incen,causa,superficie,fecha_ini,fecha_ter,tipo_dato,id_pixel,id_pixel_incendio,geometry
0,0,2022-02-02,0,0.1324,0.1278,0.1418,0.1498,0.2619,0.2929,0.2298,...,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,1,2021-2022-3-1,POINT (269650.000 6265890.000)
1,1,2022-02-02,1,0.1313,0.1252,0.1425,0.1383,0.2820,0.2676,0.2038,...,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,2,2021-2022-3-2,POINT (269930.000 6264130.000)
2,2,2022-02-02,2,0.1388,0.1322,0.1465,0.1517,0.2457,0.2914,0.2357,...,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,3,2021-2022-3-3,POINT (268650.000 6263950.000)
3,3,2022-02-02,3,0.1325,0.1284,0.1394,0.1420,0.2357,0.2532,0.2015,...,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,4,2021-2022-3-4,POINT (271090.000 6264110.000)
4,4,2022-02-02,4,0.1350,0.1376,0.1542,0.1680,0.2580,0.3237,0.2530,...,5601,335-SAN JUAN DE HUINCA,1.9.1. Corte de cable eléctrico por caída de r...,310.116,2022-02-23,2022-03-02,zona_incendio,5,2021-2022-3-5,POINT (267790.000 6263770.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202259,202259,2020-12-24,2985,0.0477,0.0715,0.1014,0.1504,0.2394,0.3648,0.2752,...,5801,151 - LA FRAGUA,2.1.11. Otros intencionales no clasificados,348.660,2020-12-15,2020-12-23,zona_random,53339,2020-2021-11-53339,POINT (337370.000 6413410.000)
202260,202260,2020-12-24,2986,0.0613,0.0895,0.1366,0.2058,0.2512,0.2692,0.2359,...,5801,151 - LA FRAGUA,2.1.11. Otros intencionales no clasificados,348.660,2020-12-15,2020-12-23,zona_random,53340,2020-2021-11-53340,POINT (355550.000 6414050.000)
202261,202261,2020-12-24,2988,0.0606,0.0809,0.1080,0.1515,0.2329,0.3211,0.2456,...,5801,151 - LA FRAGUA,2.1.11. Otros intencionales no clasificados,348.660,2020-12-15,2020-12-23,zona_random,53341,2020-2021-11-53341,POINT (299270.000 6418430.000)
202262,202262,2020-12-24,2994,0.0494,0.0539,0.0757,0.0869,0.2506,0.2643,0.1827,...,5801,151 - LA FRAGUA,2.1.11. Otros intencionales no clasificados,348.660,2020-12-15,2020-12-23,zona_random,53342,2020-2021-11-53342,POINT (291270.000 6438610.000)


In [5]:
df.crs

<Projected CRS: EPSG:32719>
Name: WGS 84 / UTM zone 19S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 72°W and 66°W, southern hemisphere between 80°S and equator, onshore and offshore. Argentina. Bolivia. Brazil. Chile. Colombia. Peru.
- bounds: (-72.0, -80.0, -66.0, 0.0)
Coordinate Operation:
- name: UTM zone 19S
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
# load data pixeles
# df = pd.read_csv('muestra_datos_consolidados_v2.csv')

# # editando geometria
# df['geometry'] = df['geometry'].apply(wkt.loads)
# df = gpd.GeoDataFrame(df, geometry='geometry')
# df.crs = "EPSG:32719"

In [6]:
# load .shp
# lista_gdfs = []
dct_gdfs = {
    "gdf": [],
    "ruta": []
}

for directorio_raiz, directorios, archivos in os.walk('../shp'):
    for i, archivo in enumerate(archivos):
        if archivo.endswith('.shp'):            # Solo considerar archivos con extensión .shp
            ruta_archivo = os.path.join(directorio_raiz, archivo)
            print(i, ruta_archivo)
            gdf = gpd.read_file(ruta_archivo)   # Cargar el Shapefile
            # lista_gdfs.append(gdf)
            dct_gdfs["gdf"].append(gdf)
            dct_gdfs["ruta"].append(ruta_archivo)

# print(lista_gdfs)

1 ../shp/red_hidrografica/Red_Hidrografica.shp
5 ../shp/red_vial/redvial2019.shp
6 ../shp/areas_silverstres_protegidas/snaspe.shp
5 ../shp/areas_urbanas/Areas_Pobladas.shp


In [7]:
for k in dct_gdfs:
    print(k, len(dct_gdfs[k]))

for i, r in enumerate(dct_gdfs["ruta"]):
    print(i, r)

gdf 4
ruta 4
0 ../shp/red_hidrografica/Red_Hidrografica.shp
1 ../shp/red_vial/redvial2019.shp
2 ../shp/areas_silverstres_protegidas/snaspe.shp
3 ../shp/areas_urbanas/Areas_Pobladas.shp


In [8]:
# lista_gdfs[0]
dct_gdfs["gdf"][0]

,objectid,fid_drenes,cod_comuna,shape_leng,st_length_,Nombre,Dren_Tipo,Region,Provincia,Comuna,geometry
0,1,18231,1240,592.367618,900.245347,Estero Triple,Estero,Región de Magallanes y Antártica Chilena,Ultima Esperanza,Natales,"MULTILINESTRING ((-8322721.754 -6260445.190, -..."
1,2,18231,1240,4395.679933,6684.114801,Estero Triple,Estero,Región de Magallanes y Antártica Chilena,Ultima Esperanza,Natales,"MULTILINESTRING ((-8322601.918 -6260627.679, -..."
2,3,18554,1240,1175.704231,1798.390825,Estero Dona Ana,Estero,Región de Magallanes y Antártica Chilena,Ultima Esperanza,Natales,"MULTILINESTRING ((-8361306.123 -6322289.680, -..."
3,4,18554,1240,4331.636599,6627.655554,Estero Dona Ana,Estero,Región de Magallanes y Antártica Chilena,Ultima Esperanza,Natales,"MULTILINESTRING ((-8360921.482 -6322265.853, -..."
4,5,18582,1240,1725.164577,2640.838605,Estero Dona Ana,Estero,Región de Magallanes y Antártica Chilena,Ultima Esperanza,Natales,"MULTILINESTRING ((-8364978.242 -6328874.799, -..."
...,...,...,...,...,...,...,...,...,...,...,...
88269,138319,122794,7109,4276.995508,4947.495351,NaN,Quebrada,Región del Maule,Talca,San Clemente,"LINESTRING (-7853922.568 -4277603.344, -785386..."
88270,138320,122794,7109,4276.995508,3187.539299,NaN,Quebrada,Región del Maule,Talca,San Clemente,"LINESTRING (-7850680.960 -4278484.014, -785068..."
88271,138321,122794,7109,4276.995508,11366.736196,Arroyo de la Hora,Quebrada,Región del Maule,Talca,San Clemente,"LINESTRING (-7845776.304 -4280191.076, -784583..."
88272,138322,122794,7109,4276.995508,12471.004936,NaN,Quebrada,Región del Maule,Talca,San Clemente,"LINESTRING (-7879726.667 -4267773.752, -787957..."


NO SIRVEN:

- shp\masas_lacustres\masas_lacustres.shp
- shp\red_ferroviaria\Red_ferroviaria.shp 


In [9]:
# areas_silvestres_protegidas = lista_gdfs[0][lista_gdfs[0]['Region'] == 'Región de Valparaíso']
# red_hidrografica = lista_gdfs[5][lista_gdfs[5]['Region'] == 'Región de Valparaíso']

areas_silvestres_protegidas = dct_gdfs["gdf"][2][dct_gdfs["gdf"][2]['Region'] == 'Región de Valparaíso']
red_hidrografica = dct_gdfs["gdf"][0][dct_gdfs["gdf"][0]['Region'] == 'Región de Valparaíso']

In [11]:
# areas_silvestres_protegidas
red_hidrografica

,objectid,fid_drenes,cod_comuna,shape_leng,st_length_,Nombre,Dren_Tipo,Region,Provincia,Comuna,geometry
36112,39077,113027,5704,2987.979247,3552.987169,Quebrada Las Tinajas,Quebrada,Región de Valparaíso,San Felipe,Panquehue,"LINESTRING (-7877144.656 -3867570.196, -787739..."
36113,39078,113052,5704,708.648641,844.166351,Quebrada Las Jarillas,Quebrada,Región de Valparaíso,San Felipe,Panquehue,"LINESTRING (-7881141.728 -3869289.909, -788142..."
36114,39079,113053,5704,1850.865212,2202.873322,Quebrada El Aguila,Quebrada,Región de Valparaíso,San Felipe,Panquehue,"LINESTRING (-7879378.400 -3868559.599, -787945..."
36115,39080,113064,5704,4343.689316,5167.816039,Quebrada Las Jarillas,Quebrada,Región de Valparaíso,San Felipe,Panquehue,"LINESTRING (-7876565.688 -3870646.530, -787664..."
36116,39081,113072,5704,3442.094197,4094.581924,Quebrada Mala,Quebrada,Región de Valparaíso,San Felipe,Panquehue,"LINESTRING (-7880804.166 -3869771.608, -788064..."
...,...,...,...,...,...,...,...,...,...,...,...
87330,125419,113149,5304,10.058718,689.122095,Río Aconcagua,Río,Región de Valparaíso,Los Andes,San Esteban,"LINESTRING (-7881627.926 -3861386.141, -788161..."
87331,125420,113149,5304,10.058718,260.556075,Río Aconcagua,Río,Región de Valparaíso,Los Andes,San Esteban,"LINESTRING (-7880249.872 -3860526.420, -788025..."
87332,125421,113149,5304,10.058718,558.562104,Río Aconcagua,Río,Región de Valparaíso,Los Andes,San Esteban,"LINESTRING (-7879985.241 -3860418.882, -787995..."
87333,125422,113149,5304,10.058718,445.216052,Río Aconcagua,Río,Región de Valparaíso,Los Andes,San Esteban,"LINESTRING (-7879747.932 -3860256.061, -787969..."


In [12]:
# AREAS URBANAS VALPO

areas_urbanas = dct_gdfs["gdf"][3]

aux = red_hidrografica['Comuna'].unique()  # no esta rapa nui ni juan fernandez!
print(aux)
aux[1] = 'La Calera'

# areas_urbanas = lista_gdfs[1][lista_gdfs[1]['comuna'].isin(aux)]
areas_urbanas = areas_urbanas[areas_urbanas['comuna'].isin(aux)]
areas_urbanas

['Panquehue' 'Calera' 'La Cruz' 'Rinconada' 'San Felipe' 'Llaillay'
 'Petorca' 'La Ligua' 'Cabildo' 'Olmué' 'Viña del Mar' 'Hijuelas'
 'Limache' 'Zapallar' 'Nogales' 'Catemu' 'Puchuncaví' 'Putaendo'
 'Quillota' 'Concón' 'Papudo' 'Quintero' 'Algarrobo' 'Santa María'
 'Calle Larga' 'Villa Alemana' 'Quilpué' 'Valparaíso' 'Casablanca'
 'El Quisco' 'El Tabo' 'Cartagena' 'Santo Domingo' 'San Antonio'
 'San Esteban' 'Los Andes']


,objectid,comuna,st_area_sh,st_length_,Localidad,Entidad,geometry
158,194,Casablanca,4.021006e+05,4018.578534,Quintay,Pueblo,"POLYGON ((-7981495.070 -3920944.993, -7981491...."
195,193,Algarrobo,2.950864e+05,3218.526174,El Yeco,Pueblo,"POLYGON ((-7976028.683 -3936619.030, -7975974...."
198,195,Quillota,8.630518e+05,7120.219432,San Pedro,Pueblo,"POLYGON ((-7934669.810 -3887276.826, -7934659...."
199,196,Zapallar,7.070413e+05,3922.796333,Catapilco,Pueblo,"POLYGON ((-7934702.603 -3837693.576, -7934687...."
202,198,Cabildo,2.800146e+06,11954.646432,Cabildo,Ciudad,"POLYGON ((-7911753.269 -3819690.485, -7911593...."
203,202,Putaendo,1.594487e+06,6567.120103,Putaendo,Ciudad,"POLYGON ((-7871991.423 -3846942.932, -7872231...."
204,200,Petorca,1.562601e+06,9892.170654,Petorca,Pueblo,"POLYGON ((-7897938.923 -3796707.681, -7897927...."
205,201,La Ligua,5.086825e+06,17968.319378,La Ligua,Ciudad,"POLYGON ((-7929150.053 -3823283.859, -7929152...."
207,203,San Felipe,1.250840e+07,27539.938175,San Felipe,Ciudad,"POLYGON ((-7875156.498 -3860508.871, -7874746...."
208,207,Llaillay,3.808810e+06,16938.978687,Llaillay,Ciudad,"POLYGON ((-7898745.990 -3874524.009, -7899657...."


In [13]:
# red_vial_2019 = lista_gdfs[6][lista_gdfs[6]['Cod_Region']==5]
red_vial_2019 = dct_gdfs["gdf"][1][dct_gdfs["gdf"][1]['Cod_Region']==5]
red_vial_2019

,shape_leng,st_length_,Nom_Ruta,Rol_Mop,Clase_Ruta,Tipo_Carpe,Catego,Cod_Region,geometry
13,9603.351160,937.122567,NaN,E-105-G,3,Tierra,Rural,5,"LINESTRING (339118.491 6353577.074, 339155.396..."
348,33.531712,235.546826,NaN,NaN,9,NaN,Urbano,5,"LINESTRING (317043.464 6371100.066, 317109.934..."
349,33.531712,48.546574,NaN,NaN,9,NaN,Urbano,5,"LINESTRING (316634.322 6371177.909, 316637.778..."
350,33.531712,52.984459,NaN,NaN,9,NaN,Urbano,5,"LINESTRING (316589.890 6371172.969, 316594.894..."
351,33.531712,63.491028,NaN,NaN,9,NaN,Urbano,5,"LINESTRING (316544.705 6371167.946, 316551.152..."
...,...,...,...,...,...,...,...,...,...
434783,1885.731382,2026.357795,NaN,E-895,3,Pavimento,Rural,5,"LINESTRING (351460.685 6362005.815, 351452.691..."
434784,755.955997,755.955997,Pascual Baburizza,NaN,3,NaN,Rural,5,"LINESTRING (351291.864 6363100.215, 351293.748..."
434785,0.000000,650.347086,Cruce Ruta 57 - Badén La Caldera - Cruce E-909,S/R-E-917,3,NaN,Urbano,5,"LINESTRING (349063.448 6360443.645, 349078.885..."
434786,0.000000,571.464714,Cruce Ruta 57 - Badén La Caldera - Cruce E-909,S/R-E-917,3,NaN,Rural,5,"LINESTRING (348982.469 6359902.944, 348982.789..."


#### Calculando distancias:

In [15]:
# masas_lacustres_sur = dct_gdfs["gdf"][5]

# nom_vars_list = ['d_areas_silvestres_protegidas', 'd_red_hidrografica', 'd_areas_urbanas', 'd_red_vial', 'd_masas_lacustres_sur']
nom_vars_list = ['d_areas_silvestres_protegidas', 'd_red_hidrografica', 'd_areas_urbanas', 'd_red_vial']
# df_sh_list = [areas_silvestres_protegidas, red_hidrografica, areas_urbanas, red_vial_2019, masas_lacustres_sur]
df_sh_list = [areas_silvestres_protegidas, red_hidrografica, areas_urbanas, red_vial_2019]

df_dist = calcular_distancia(df, nom_vars_list, df_sh_list)
df_dist

revisando 0----
revisando 1----


/home/hugosoto/anaconda3/envs/ml/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/hugosoto/anaconda3/envs/ml/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


revisando 2----


/home/hugosoto/anaconda3/envs/ml/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/hugosoto/anaconda3/envs/ml/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


revisando 3----


/home/hugosoto/anaconda3/envs/ml/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/hugosoto/anaconda3/envs/ml/lib/python3.10/site-packages/geopandas/geoseries.py:645: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  result = super().apply(func, convert_dtype=convert_dtype, args=args, **kwargs)


,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,fecha_ini,fecha_ter,tipo_dato,id_pixel,id_pixel_incendio,geometry,d_areas_silvestres_protegidas,d_red_hidrografica,d_areas_urbanas,d_red_vial
0,0,2022-02-02,0,0.1324,0.1278,0.1418,0.1498,0.2619,0.2929,0.2298,...,2022-02-23,2022-03-02,zona_incendio,1,2021-2022-3-1,POINT (269650.000 6265890.000),17016.080381,351.897310,12267.555944,2271.689885
1,1,2022-02-02,1,0.1313,0.1252,0.1425,0.1383,0.2820,0.2676,0.2038,...,2022-02-23,2022-03-02,zona_incendio,2,2021-2022-3-2,POINT (269930.000 6264130.000),16811.721774,523.761474,13747.869632,2304.066784
2,2,2022-02-02,2,0.1388,0.1322,0.1465,0.1517,0.2457,0.2914,0.2357,...,2022-02-23,2022-03-02,zona_incendio,3,2021-2022-3-3,POINT (268650.000 6263950.000),15523.777524,645.140872,13093.957620,3572.997865
3,3,2022-02-02,3,0.1325,0.1284,0.1394,0.1420,0.2357,0.2532,0.2015,...,2022-02-23,2022-03-02,zona_incendio,4,2021-2022-3-4,POINT (271090.000 6264110.000),17941.154099,51.029235,14545.973688,1151.776263
4,4,2022-02-02,4,0.1350,0.1376,0.1542,0.1680,0.2580,0.3237,0.2530,...,2022-02-23,2022-03-02,zona_incendio,5,2021-2022-3-5,POINT (267790.000 6263770.000),14645.361310,155.051941,12627.989806,4434.598892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202259,202259,2020-12-24,2985,0.0477,0.0715,0.1014,0.1504,0.2394,0.3648,0.2752,...,2020-12-15,2020-12-23,zona_random,53339,2020-2021-11-53339,POINT (337370.000 6413410.000),65288.415231,532.184821,23820.932821,2344.264307
202260,202260,2020-12-24,2986,0.0613,0.0895,0.1366,0.2058,0.2512,0.2692,0.2359,...,2020-12-15,2020-12-23,zona_random,53340,2020-2021-11-53340,POINT (355550.000 6414050.000),75389.173713,648.145936,29498.664489,11918.168975
202261,202261,2020-12-24,2988,0.0606,0.0809,0.1080,0.1515,0.2329,0.3211,0.2456,...,2020-12-15,2020-12-23,zona_random,53341,2020-2021-11-53341,POINT (299270.000 6418430.000),39807.426376,369.686623,9887.002430,1886.822519
202262,202262,2020-12-24,2994,0.0494,0.0539,0.0757,0.0869,0.2506,0.2643,0.1827,...,2020-12-15,2020-12-23,zona_random,53342,2020-2021-11-53342,POINT (291270.000 6438610.000),51400.147186,806.570891,26575.184516,5117.981943


In [18]:
# del lista_gdfs
df_dist.describe()

,Unnamed: 0,dim_0,coastal,blue,green,red,nir08,swir16,swir22,y,...,es_incendio,codreg,codprov,codcom,superficie,id_pixel,d_areas_silvestres_protegidas,d_red_hidrografica,d_areas_urbanas,d_red_vial
count,202264.000000,202264.000000,193141.000000,193131.000000,193150.000000,193166.000000,193225.000000,193275.000000,193288.000000,2.022640e+05,...,202264.000000,202264.0,202264.000000,202264.000000,202264.000000,202264.000000,202264.000000,202264.000000,202264.000000,202264.000000
mean,101131.500000,1054.582719,0.073176,0.081635,0.102313,0.122066,0.220589,0.237672,0.182605,6.349591e+06,...,0.186069,5.0,54.971157,5499.445116,673.936065,28127.750672,28914.446187,595.807822,12101.218407,3660.768261
std,58388.731761,838.303185,0.088142,0.084791,0.082373,0.085835,0.084085,0.083302,0.071954,5.367037e+04,...,0.389163,0.0,2.698365,269.740735,787.159590,15837.248239,25642.503631,522.761471,9778.854534,5641.645818
min,0.000000,0.000000,0.000000,0.000000,0.000100,0.000100,0.000000,0.000000,0.000100,6.240110e+06,...,0.000000,5.0,51.000000,5101.000000,210.938000,1.000000,0.000000,0.003709,0.000000,0.021707
25%,50565.750000,395.000000,0.035600,0.042900,0.060900,0.070500,0.174100,0.181200,0.133200,6.323690e+06,...,0.000000,5.0,51.000000,5105.000000,252.660000,15346.000000,8891.868303,208.988533,5073.394549,632.727573
50%,101131.500000,793.000000,0.051300,0.062400,0.084500,0.109500,0.214200,0.230300,0.178700,6.348610e+06,...,0.000000,5.0,56.000000,5601.000000,336.354000,25095.000000,20525.328212,454.334475,9461.789911,1715.167387
75%,151697.250000,1643.000000,0.076100,0.089300,0.116300,0.149700,0.255400,0.288700,0.223600,6.392630e+06,...,0.000000,5.0,58.000000,5801.000000,563.156816,42852.000000,42520.198232,838.630503,17854.301485,4231.745232
max,202263.000000,2999.000000,0.998600,0.999700,0.998400,0.999600,0.999900,0.985000,0.997400,6.454910e+06,...,1.000000,5.0,58.000000,5804.000000,3403.060000,53343.000000,100918.022215,5504.871457,56364.870903,40080.936512


In [19]:
df_dist.shape, df.shape

((202264, 35), (202264, 35))

### Calculando índices relevantes

In [20]:
# ndvi
df["ndvi"] = (df['nir08'] - df['red']) / (df['nir08'] + df['red'])
 
# evi
df["evi"] = 2.5 * ((df['nir08'] - df['red']) / (df['nir08'] + 6 * df['red'] - 7.5 * df['blue'] + 1))
 
# ndwi
df["ndwi"] = (df['nir08'] - df['swir16']) / (df['nir08'] + df['swir16'])
 
# - TODO: Índice Normalizado de Área Quemada (NBR)

In [21]:
df.dtypes

Unnamed: 0                          int64
time                               object
dim_0                               int64
coastal                           float64
blue                              float64
green                             float64
red                               float64
nir08                             float64
swir16                            float64
swir22                            float64
y                                 float64
x                                 float64
spatial_ref                         int64
es_incendio                         int64
id_incendio                        object
temporada                          object
region                             object
provincia                          object
comuna                             object
codreg                              int64
codprov                             int64
codcom                              int64
nom_incen                          object
causa                             

### Datos climáticos con Google Earth Engine

In [22]:
import ee
ee.Authenticate()

True

In [23]:
# Inicializar Earth Engine
ee.Initialize()

In [24]:
df

,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,id_pixel,id_pixel_incendio,geometry,d_areas_silvestres_protegidas,d_red_hidrografica,d_areas_urbanas,d_red_vial,ndvi,evi,ndwi
0,0,2022-02-02,0,0.1324,0.1278,0.1418,0.1498,0.2619,0.2929,0.2298,...,1,2021-2022-3-1,POINT (269650.000 6265890.000),17016.080381,351.897310,12267.555944,2271.689885,0.272286,0.233114,-0.055876
1,1,2022-02-02,1,0.1313,0.1252,0.1425,0.1383,0.2820,0.2676,0.2038,...,2,2021-2022-3-2,POINT (269930.000 6264130.000),16811.721774,523.761474,13747.869632,2304.066784,0.341899,0.306318,0.026201
2,2,2022-02-02,2,0.1388,0.1322,0.1465,0.1517,0.2457,0.2914,0.2357,...,3,2021-2022-3-3,POINT (268650.000 6263950.000),15523.777524,645.140872,13093.957620,3572.997865,0.236537,0.201821,-0.085087
3,3,2022-02-02,3,0.1325,0.1284,0.1394,0.1420,0.2357,0.2532,0.2015,...,4,2021-2022-3-4,POINT (271090.000 6264110.000),17941.154099,51.029235,14545.973688,1151.776263,0.248080,0.208278,-0.035795
4,4,2022-02-02,4,0.1350,0.1376,0.1542,0.1680,0.2580,0.3237,0.2530,...,5,2021-2022-3-5,POINT (267790.000 6263770.000),14645.361310,155.051941,12627.989806,4434.598892,0.211268,0.182334,-0.112945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202259,202259,2020-12-24,2985,0.0477,0.0715,0.1014,0.1504,0.2394,0.3648,0.2752,...,53339,2020-2021-11-53339,POINT (337370.000 6413410.000),65288.415231,532.184821,23820.932821,2344.264307,0.228322,0.138582,-0.207547
202260,202260,2020-12-24,2986,0.0613,0.0895,0.1366,0.2058,0.2512,0.2692,0.2359,...,53340,2020-2021-11-53340,POINT (355550.000 6414050.000),75389.173713,648.145936,29498.664489,11918.168975,0.099344,0.062543,-0.034589
202261,202261,2020-12-24,2988,0.0606,0.0809,0.1080,0.1515,0.2329,0.3211,0.2456,...,53341,2020-2021-11-53341,POINT (299270.000 6418430.000),39807.426376,369.686623,9887.002430,1886.822519,0.211759,0.132560,-0.159206
202262,202262,2020-12-24,2994,0.0494,0.0539,0.0757,0.0869,0.2506,0.2643,0.1827,...,53342,2020-2021-11-53342,POINT (291270.000 6438610.000),51400.147186,806.570891,26575.184516,5117.981943,0.485037,0.299214,-0.026607


In [25]:
df_4326 = df.to_crs("EPSG:4326")
df_4326['time2'] = [item.split(" ", 1)[0] for item in df_4326['time']]

# Obtenemos las fechas de captura de cada uno, para realizar un posterior filtro en las imagenes recolectadas
DATES = df_4326.time.unique()
DATES = [item.split(" ", 1)[0] for item in DATES]
print(len(DATES))
print(DATES)

inicio_datetime = min(DATES)
fin_datetime = max(DATES)

53
['2022-02-02', '2022-02-07', '2022-02-12', '2022-02-17', '2022-02-22', '2022-02-27', '2022-03-04', '2021-07-02', '2021-07-07', '2021-07-17', '2021-07-22', '2021-07-27', '2021-08-01', '2022-03-09', '2017-01-19', '2017-01-29', '2017-02-18', '2017-02-28', '2017-03-10', '2017-03-20', '2018-12-10', '2018-12-15', '2018-12-20', '2018-12-30', '2019-01-04', '2019-01-14', '2019-01-19', '2019-01-24', '2019-01-29', '2019-02-03', '2019-01-09', '2022-12-04', '2022-12-09', '2022-12-14', '2022-12-19', '2022-12-24', '2023-05-03', '2023-05-08', '2023-05-18', '2022-12-29', '2022-10-30', '2022-11-04', '2022-11-19', '2020-12-24', '2020-12-29', '2021-01-03', '2021-01-13', '2021-01-18', '2020-11-24', '2020-11-29', '2020-12-04', '2020-12-09', '2020-12-19']


In [26]:
sorted(DATES)

['2017-01-19',
 '2017-01-29',
 '2017-02-18',
 '2017-02-28',
 '2017-03-10',
 '2017-03-20',
 '2018-12-10',
 '2018-12-15',
 '2018-12-20',
 '2018-12-30',
 '2019-01-04',
 '2019-01-09',
 '2019-01-14',
 '2019-01-19',
 '2019-01-24',
 '2019-01-29',
 '2019-02-03',
 '2020-11-24',
 '2020-11-29',
 '2020-12-04',
 '2020-12-09',
 '2020-12-19',
 '2020-12-24',
 '2020-12-29',
 '2021-01-03',
 '2021-01-13',
 '2021-01-18',
 '2021-07-02',
 '2021-07-07',
 '2021-07-17',
 '2021-07-22',
 '2021-07-27',
 '2021-08-01',
 '2022-02-02',
 '2022-02-07',
 '2022-02-12',
 '2022-02-17',
 '2022-02-22',
 '2022-02-27',
 '2022-03-04',
 '2022-03-09',
 '2022-10-30',
 '2022-11-04',
 '2022-11-19',
 '2022-12-04',
 '2022-12-09',
 '2022-12-14',
 '2022-12-19',
 '2022-12-24',
 '2022-12-29',
 '2023-05-03',
 '2023-05-08',
 '2023-05-18']

In [27]:
df_4326

,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,id_pixel_incendio,geometry,d_areas_silvestres_protegidas,d_red_hidrografica,d_areas_urbanas,d_red_vial,ndvi,evi,ndwi,time2
0,0,2022-02-02,0,0.1324,0.1278,0.1418,0.1498,0.2619,0.2929,0.2298,...,2021-2022-3-1,POINT (-71.48600 -33.72202),17016.080381,351.897310,12267.555944,2271.689885,0.272286,0.233114,-0.055876,2022-02-02
1,1,2022-02-02,1,0.1313,0.1252,0.1425,0.1383,0.2820,0.2676,0.2038,...,2021-2022-3-2,POINT (-71.48344 -33.73794),16811.721774,523.761474,13747.869632,2304.066784,0.341899,0.306318,0.026201,2022-02-02
2,2,2022-02-02,2,0.1388,0.1322,0.1465,0.1517,0.2457,0.2914,0.2357,...,2021-2022-3-3,POINT (-71.49729 -33.73928),15523.777524,645.140872,13093.957620,3572.997865,0.236537,0.201821,-0.085087,2022-02-02
3,3,2022-02-02,3,0.1325,0.1284,0.1394,0.1420,0.2357,0.2532,0.2015,...,2021-2022-3-4,POINT (-71.47093 -33.73837),17941.154099,51.029235,14545.973688,1151.776263,0.248080,0.208278,-0.035795,2022-02-02
4,4,2022-02-02,4,0.1350,0.1376,0.1542,0.1680,0.2580,0.3237,0.2530,...,2021-2022-3-5,POINT (-71.50662 -33.74072),14645.361310,155.051941,12627.989806,4434.598892,0.211268,0.182334,-0.112945,2022-02-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202259,202259,2020-12-24,2985,0.0477,0.0715,0.1014,0.1504,0.2394,0.3648,0.2752,...,2020-2021-11-53339,POINT (-70.72929 -32.40451),65288.415231,532.184821,23820.932821,2344.264307,0.228322,0.138582,-0.207547,2020-12-24
202260,202260,2020-12-24,2986,0.0613,0.0895,0.1366,0.2058,0.2512,0.2692,0.2359,...,2020-2021-11-53340,POINT (-70.53594 -32.40125),75389.173713,648.145936,29498.664489,11918.168975,0.099344,0.062543,-0.034589,2020-12-24
202261,202261,2020-12-24,2988,0.0606,0.0809,0.1080,0.1515,0.2329,0.3211,0.2456,...,2020-2021-11-53341,POINT (-71.13313 -32.35305),39807.426376,369.686623,9887.002430,1886.822519,0.211759,0.132560,-0.159206,2020-12-24
202262,202262,2020-12-24,2994,0.0494,0.0539,0.0757,0.0869,0.2506,0.2643,0.1827,...,2020-2021-11-53342,POINT (-71.21367 -32.16967),51400.147186,806.570891,26575.184516,5117.981943,0.485037,0.299214,-0.026607,2020-12-24


In [28]:
df_4326[df_4326["time"] == "2017-01-19"]

,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,id_pixel_incendio,geometry,d_areas_silvestres_protegidas,d_red_hidrografica,d_areas_urbanas,d_red_vial,ndvi,evi,ndwi,time2
16000,16000,2017-01-19,0,0.0678,0.0772,0.1017,0.1298,0.1971,0.2933,0.2498,...,2016-2017-70-2912,POINT (-71.34064 -32.98692),18523.448577,483.321736,5109.589007,1197.793688,0.205873,0.120445,-0.196166,2017-01-19
16001,16001,2017-01-19,1,0.0621,0.0641,0.0844,0.1105,0.1735,0.2831,0.2450,...,2016-2017-70-2913,POINT (-71.33389 -32.99047),17909.929836,580.526521,4474.306333,468.252766,0.221831,0.116172,-0.240035,2017-01-19
16002,16002,2017-01-19,2,0.0558,0.0521,0.0634,0.0731,0.1039,0.1992,0.2016,...,2016-2017-70-2914,POINT (-71.34390 -32.98902),18837.420114,280.141143,4920.093976,1395.783118,0.174011,0.066855,-0.314418,2017-01-19
16003,16003,2017-01-19,3,0.0633,0.0638,0.0872,0.1014,0.2587,0.2481,0.1669,...,2016-2017-70-2915,POINT (-71.32185 -33.00495),16958.084246,27.188025,3434.478623,523.143517,0.436823,0.283199,0.020916,2017-01-19
16004,16004,2017-01-19,4,0.0483,0.0437,0.0524,0.0590,0.0818,0.1260,0.1236,...,2016-2017-70-2916,POINT (-71.33617 -33.01207),17658.453086,383.692629,2789.300024,963.739367,0.161932,0.051442,-0.212705,2017-01-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127036,127036,2017-01-19,2990,0.0479,0.0530,0.0728,0.0927,0.1953,0.2844,0.2116,...,2016-2017-143-38308,POINT (-71.23305 -32.20775),46805.277300,676.944997,23399.863778,7249.233066,0.356250,0.189439,-0.185741,2017-01-19
127037,127037,2017-01-19,2991,0.0542,0.0599,0.0907,0.1299,0.1930,0.1708,0.1557,...,2016-2017-143-38309,POINT (-70.24001 -32.68521),77688.984485,754.442758,25468.947916,19784.579536,0.195417,0.103568,0.061023,2017-01-19
127038,127038,2017-01-19,2993,0.0233,0.0190,0.0400,0.0307,0.2177,0.1181,0.0574,...,2016-2017-143-38310,POINT (-71.64099 -33.17766),8065.795933,775.446394,5192.122382,3219.220060,0.752818,0.371209,0.296605,2017-01-19
127039,127039,2017-01-19,2995,0.0705,0.0807,0.1127,0.1472,0.2096,0.2954,0.2399,...,2016-2017-143-38311,POINT (-71.09379 -32.18542),55923.078209,530.384591,15479.317492,610.483236,0.174888,0.104870,-0.169901,2017-01-19


In [29]:
list_df = []

for i, fecha in enumerate(DATES):
    print(f"Extrayendo info para el día: {(i, fecha)}")
    
    # Definir la colección de imágenes
    image_collection = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR') \
        .filterDate(fecha) \
        .select([
            'temperature_2m',
            'temperature_2m_min',
            'temperature_2m_max',
            'dewpoint_temperature_2m',
            'dewpoint_temperature_2m_min',
            'dewpoint_temperature_2m_max',
            'soil_temperature_level_1',
            'soil_temperature_level_1_min',
            'soil_temperature_level_1_max',
            'volumetric_soil_water_layer_1',
            'volumetric_soil_water_layer_1_min',
            'volumetric_soil_water_layer_1_max',
            'u_component_of_wind_10m',
            'u_component_of_wind_10m_min',
            'u_component_of_wind_10m_max',
            'v_component_of_wind_10m',
            'v_component_of_wind_10m_min',
            'v_component_of_wind_10m_max',
            'total_precipitation_sum',
            'total_precipitation_min',
            'total_precipitation_max'
        ])
        
    # Realizando estimaciones: 
    df_aux = calculate_weather(image_collection.first(), fecha, df_4326)
              
    list_df.append(df_aux)
    
data_con_gee = pd.concat(list_df, ignore_index = True)
        

Extrayendo info para el día: (0, '2022-02-02')
Extrayendo info para el día: (1, '2022-02-07')
Extrayendo info para el día: (2, '2022-02-12')
Extrayendo info para el día: (3, '2022-02-17')
Extrayendo info para el día: (4, '2022-02-22')
Extrayendo info para el día: (5, '2022-02-27')
Extrayendo info para el día: (6, '2022-03-04')
Extrayendo info para el día: (7, '2021-07-02')
Extrayendo info para el día: (8, '2021-07-07')
Extrayendo info para el día: (9, '2021-07-17')
Extrayendo info para el día: (10, '2021-07-22')
Extrayendo info para el día: (11, '2021-07-27')
Extrayendo info para el día: (12, '2021-08-01')
Extrayendo info para el día: (13, '2022-03-09')
Extrayendo info para el día: (14, '2017-01-19')
Extrayendo info para el día: (15, '2017-01-29')
Extrayendo info para el día: (16, '2017-02-18')
Extrayendo info para el día: (17, '2017-02-28')
Extrayendo info para el día: (18, '2017-03-10')
Extrayendo info para el día: (19, '2017-03-20')
Extrayendo info para el día: (20, '2018-12-10')
Ex

In [30]:
data_con_gee

,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,total_precipitation_sum,u_component_of_wind_10m,u_component_of_wind_10m_max,u_component_of_wind_10m_min,v_component_of_wind_10m,v_component_of_wind_10m_max,v_component_of_wind_10m_min,volumetric_soil_water_layer_1,volumetric_soil_water_layer_1_max,volumetric_soil_water_layer_1_min
0,0,2022-02-02,0,0.1324,0.1278,0.1418,0.1498,0.2619,0.2929,0.2298,...,2.682209e-07,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817
1,1,2022-02-02,1,0.1313,0.1252,0.1425,0.1383,0.2820,0.2676,0.2038,...,2.682209e-07,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817
2,2,2022-02-02,2,0.1388,0.1322,0.1465,0.1517,0.2457,0.2914,0.2357,...,2.682209e-07,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817
3,3,2022-02-02,3,0.1325,0.1284,0.1394,0.1420,0.2357,0.2532,0.2015,...,2.682209e-07,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817
4,4,2022-02-02,4,0.1350,0.1376,0.1542,0.1680,0.2580,0.3237,0.2530,...,2.682209e-07,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202259,201000,2020-12-19,2985,0.0547,0.0712,0.1033,0.1552,0.2478,0.3660,0.2628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202260,201001,2020-12-19,2986,0.0785,0.0973,0.1426,0.2115,0.2704,0.2834,0.2419,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202261,201002,2020-12-19,2988,0.0538,0.0695,0.0983,0.1443,0.2249,0.3125,0.2389,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202262,201003,2020-12-19,2994,0.0468,0.0439,0.0708,0.0812,0.2504,0.2722,0.1827,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
data_con_gee.dtypes

Unnamed: 0                              int64
time                                   object
dim_0                                   int64
coastal                               float64
blue                                  float64
green                                 float64
red                                   float64
nir08                                 float64
swir16                                float64
swir22                                float64
y                                     float64
x                                     float64
spatial_ref                             int64
es_incendio                             int64
id_incendio                            object
temporada                              object
region                                 object
provincia                              object
comuna                                 object
codreg                                  int64
codprov                                 int64
codcom                            

In [32]:
data_con_gee.to_csv('../model/data/valparaiso/data_consolidados_sentinel2_distancias_indices_era5.csv')

In [33]:
data_con_gee

,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,total_precipitation_sum,u_component_of_wind_10m,u_component_of_wind_10m_max,u_component_of_wind_10m_min,v_component_of_wind_10m,v_component_of_wind_10m_max,v_component_of_wind_10m_min,volumetric_soil_water_layer_1,volumetric_soil_water_layer_1_max,volumetric_soil_water_layer_1_min
0,0,2022-02-02,0,0.1324,0.1278,0.1418,0.1498,0.2619,0.2929,0.2298,...,2.682209e-07,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817
1,1,2022-02-02,1,0.1313,0.1252,0.1425,0.1383,0.2820,0.2676,0.2038,...,2.682209e-07,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817
2,2,2022-02-02,2,0.1388,0.1322,0.1465,0.1517,0.2457,0.2914,0.2357,...,2.682209e-07,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817
3,3,2022-02-02,3,0.1325,0.1284,0.1394,0.1420,0.2357,0.2532,0.2015,...,2.682209e-07,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817
4,4,2022-02-02,4,0.1350,0.1376,0.1542,0.1680,0.2580,0.3237,0.2530,...,2.682209e-07,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202259,201000,2020-12-19,2985,0.0547,0.0712,0.1033,0.1552,0.2478,0.3660,0.2628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202260,201001,2020-12-19,2986,0.0785,0.0973,0.1426,0.2115,0.2704,0.2834,0.2419,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202261,201002,2020-12-19,2988,0.0538,0.0695,0.0983,0.1443,0.2249,0.3125,0.2389,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202262,201003,2020-12-19,2994,0.0468,0.0439,0.0708,0.0812,0.2504,0.2722,0.1827,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
data_con_gee["anio"] = data_con_gee["id_incendio"].apply(
    lambda x: x[:4]
)
data_con_gee

,Unnamed: 0,time,dim_0,coastal,blue,green,red,nir08,swir16,swir22,...,u_component_of_wind_10m,u_component_of_wind_10m_max,u_component_of_wind_10m_min,v_component_of_wind_10m,v_component_of_wind_10m_max,v_component_of_wind_10m_min,volumetric_soil_water_layer_1,volumetric_soil_water_layer_1_max,volumetric_soil_water_layer_1_min,anio
0,0,2022-02-02,0,0.1324,0.1278,0.1418,0.1498,0.2619,0.2929,0.2298,...,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817,2021
1,1,2022-02-02,1,0.1313,0.1252,0.1425,0.1383,0.2820,0.2676,0.2038,...,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817,2021
2,2,2022-02-02,2,0.1388,0.1322,0.1465,0.1517,0.2457,0.2914,0.2357,...,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817,2021
3,3,2022-02-02,3,0.1325,0.1284,0.1394,0.1420,0.2357,0.2532,0.2015,...,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817,2021
4,4,2022-02-02,4,0.1350,0.1376,0.1542,0.1680,0.2580,0.3237,0.2530,...,1.796789,4.297897,-0.679688,-0.072233,0.688393,-0.924824,0.249864,0.249893,0.249817,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202259,201000,2020-12-19,2985,0.0547,0.0712,0.1033,0.1552,0.2478,0.3660,0.2628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
202260,201001,2020-12-19,2986,0.0785,0.0973,0.1426,0.2115,0.2704,0.2834,0.2419,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
202261,201002,2020-12-19,2988,0.0538,0.0695,0.0983,0.1443,0.2249,0.3125,0.2389,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020
202262,201003,2020-12-19,2994,0.0468,0.0439,0.0708,0.0812,0.2504,0.2722,0.1827,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020


In [46]:
data_con_gee.value_counts(subset=["anio"])

anio
2018    49630
2022    49329
2020    36264
2021    35746
2016    31295
Name: count, dtype: int64

In [47]:
n_test = 49329
n_train_val = 49630+36264+35746+31295

print(n_test, n_train_val)

49329 152935


In [48]:
(49329) / (152935+49329)

0.24388423051061978